In [2]:
import pickle
import os
import csv
import string
import librosa
import matplotlib.pyplot as plt
import numpy as np
import shutil
import soundfile as sf
import torch
from PIL import Image
from scipy.ndimage import zoom
from librosa.util import normalize
from librosa.util import fix_length
from sklearn.model_selection import train_test_split
from google.colab import drive
from google.colab import runtime
from collections import defaultdict

drive.mount('/content/drive')

Mounted at /content/drive


Paths

In [3]:
mp3_data_path = '/content/drive/My Drive/NeuraBeat/Data/fma_small/'
csv_path = '/content/drive/My Drive/NeuraBeat/tracks.csv'

Clean Filesystem

In [4]:
# !apt install ffmpeg
# !wget -O fma.zip https://os.unil.cloud.switch.ch/fma/fma_small.zip
# !unzip fma.zip
# !rm fma.zip

In [5]:
# os.remove(mp3_data_path + "README.txt")
# os.remove(mp3_data_path + "checksums")
# for root, dirs, files in os.walk(mp3_data_path):
#     for file in files:
#         file_path = os.path.join(root, file)
#         if file_path != mp3_data_path + file:
#             shutil.move(file_path, mp3_data_path)

# for root, dirs, _ in os.walk(mp3_data_path, topdown=False):
#     for folder in dirs:
#         folder_path = os.path.join(root, folder)
#         if os.path.isdir(folder_path):
#             os.rmdir(folder_path)

Create File:Genre Map

In [6]:
file_genre_map = {}  # Dictionary to store file-genre mapping
track_ids = [file_name.split('.')[0].lstrip('0') for file_name in os.listdir(mp3_data_path) if file_name.endswith('.mp3')]

# Read CSV file and create file-genre mapping
with open(csv_path, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader) # Skip headers
    next(csvreader)
    next(csvreader)
    for row in csvreader:
        if row[0] in track_ids:
            genre = row[40]
            file_genre_map[row[0]] = genre

In [7]:
# Initialize genre dist dictionary
genre_dist = {}
total_songs = 0

# Count the number of each genre
for genre in file_genre_map.values():
    if genre not in genre_dist:
        genre_dist[genre] = 0
    genre_dist[genre] += 1

# Calculate the total number of songs
total_songs = len(file_genre_map)

# Output the genre distribution and total number of songs
print("Genre distribution:")
for genre, count in genre_dist.items():
    print(f"{genre}: {count}")
print(f"Total number of songs: {total_songs}")

Genre distribution:
Hip-Hop: 1000
Pop: 1000
Folk: 1000
Experimental: 1000
Rock: 1000
International: 1000
Electronic: 1000
Instrumental: 1000
Total number of songs: 8000


Preprocess Training Data

In [9]:
data = []
labels = []
genre_counts = defaultdict(int)
max_songs_per_genre = 996
fixed_length = 162

with open(csv_path, 'r') as csvfile:
    for mp3_file in os.listdir(mp3_data_path):
        track_id = mp3_file.split('.')[0].lstrip('0')
        genre = file_genre_map[track_id]

        if genre_counts[genre] >= max_songs_per_genre:
            continue

        try:
            audio, sr = librosa.load(os.path.join(mp3_data_path, mp3_file))
            resampled_audio = librosa.resample(audio, orig_sr=sr, target_sr=22050)
            padded_audio = fix_length(resampled_audio, size=80000)
            normalized_audio = normalize(padded_audio, norm=1)
            melspec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=64, n_fft=8192, hop_length=4096)
            melspec = librosa.power_to_db(melspec, ref=np.max)
            if melspec.shape[1] > fixed_length:
              melspec = melspec[:, :fixed_length]
            else:
              melspec = np.pad(melspec, ((0, 0), (0, fixed_length - melspec.shape[1])))
            melspec_tensor = np.expand_dims(melspec, axis=0)
            melspec_tensor = torch.tensor(melspec_tensor)
            data.append(melspec_tensor)
            labels.append(genre)
            genre_counts[genre] += 1
        except Exception as e:
            print(f"Skipped corrupt file: {mp3_file}")

data = np.array(data)
labels = np.array(labels)

<ipython-input-9-045ebc6672f5>:16: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(os.path.join(mp3_data_path, mp3_file))
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-9-045ebc6672f5>:16: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(os.path.join(mp3_data_path, mp3_file))
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Skipped corrupt file: 133297.mp3


<ipython-input-9-045ebc6672f5>:16: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(os.path.join(mp3_data_path, mp3_file))
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-9-045ebc6672f5>:16: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(os.path.join(mp3_data_path, mp3_file))
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Skipped corrupt file: 108925.mp3


<ipython-input-9-045ebc6672f5>:16: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(os.path.join(mp3_data_path, mp3_file))
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Skipped corrupt file: 099134.mp3


<ipython-input-9-045ebc6672f5>:16: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(os.path.join(mp3_data_path, mp3_file))
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [10]:
training_data, val_data, training_labels, val_labels = train_test_split(data,
                                                                          labels,
                                                                          test_size=0.2,
                                                                          stratify=labels,
                                                                          random_state=42)

In [ ]:
import h5py

# Save training data to HDF5 file
with h5py.File('/content/drive/My Drive/NeuraBeat/Data/train_data_small_melspec.h5', 'w') as f:
    f.create_dataset('data', data=np.array(training_data))
    f.create_dataset('labels', data=np.array(training_labels, dtype=h5py.string_dtype(encoding='utf-8')))

# Save validation data to HDF5 file
with h5py.File('/content/drive/My Drive/NeuraBeat/Data/val_data_small_melspec.h5', 'w') as f:
    f.create_dataset('data', data=np.array(val_data))
    f.create_dataset('labels', data=np.array(val_labels, dtype=h5py.string_dtype(encoding='utf-8')))

In [ ]:
runtime.unassign()